

<p style="font-size:300%; background-color:pink; color:blue; text-align:center;line-height : 80px; margin : 0; padding : 0;">
Entrega 3 - Proyecto FInal </p>
<p style="font-size:240%; background-color:pink; color:red; text-align:center;line-height : 60px; margin : 0; padding : 0;">
    
Técnicas de Inteligencia Artificial - Visión de Máquina </p1>

<p style="font-size:200%; text-align:center; line-height : 40px;  margin-top : 0; margin-bottom : 0; "> 
  <br>Est. Brian Camilo Valencia Peña (bcvalenciap@unal.edu.co)
  <br>Est. Esteban Ladino Fajardo (eladinof@unal.edu.co)
</p>

<p style="font-size:180%; text-align:center; line-height : 30px;  margin-top : 0; margin-bottom : 0; "> <br><br>INGENIERÍA MECATRÓNICA Y ELECTRÓNICA</p>
<p style="font-size:180%; text-align:center; line-height : 30px; margin-top : 0; "> Facultad de Ingeniería</p>
<p style="font-size:160%; text-align:center; line-height : 30px; margin-top : 0; "> Universidad Nacional de Colombia Sede Bogotá</p>
<br>
<img src="../figs/Escudo_UN.png",width=100,height=100>

<p style="font-size:160%; text-align:center; line-height : 30px; margin-top : 0; "> <br>28 de Junio de 2022</p>

In [6]:
# Librerias
import cv2
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import math
from scipy import stats as st
from scipy.cluster import vq
from matplotlib.colors import ListedColormap
import time as t
print ("OpenCV Version : %s " % cv2.__version__)
from scipy import signal
from scipy import misc
from sklearn import neighbors, datasets
import getopt
import sys

%matplotlib inline


OpenCV Version : 4.5.5 


## Aplicación de algoritmos de calsificación supervisado
Detección de Cacao enfermo o sano, muestras de color rojo y amarillo. Algoritmo K-vecinos mas cercanos.


In [7]:
# importación de características y datos de procesamiento
Xpoblacional=np.load('../data/caracteristicasNormalizadasPCA.npy')
imagenesProcesadas=np.load('../data/listaImagenesProcesadas.npy')  #Arreglo de pertenencia a clases del dataset 0=sana 1=moniia 2=fito
nombreCaracteristicas=np.load('../data/nombreCaracteristicasNormalizadasPCA.npy')

print(f'Dimensiones de caractarísticas:{Xpoblacional.shape}')

#print(f'Imágenes procesadas:\n {imagenesProcesadas}')
#print(f'Nombre de características finales\n{nombreCaracteristicas}')
#print(f'caracteristicasNormalizadasPCA\n{Xpoblacional}')

#Generación de tabla con caracterisitcas y valores para cada imagen del dataset
datasetinput = pd.DataFrame(Xpoblacional, columns = nombreCaracteristicas)
datasetinput



Dimensiones de caractarísticas:(136, 17)


,Moda R,Media R,Media G,Media B,Media Gris,máximo R,máximo G,máximo B,máximo Gris,Momento HU 1,Momento HU 2,Momento HU 3,Momento HU 4,Momento HU 5,Momento HU 6,Momento HU 7,Redondez
0,0.003922,0.363162,0.348868,0.314146,0.334623,0.949020,0.941176,0.882353,0.937255,0.181905,0.105386,0.054615,0.023270,0.004066,0.015774,0.085357,0.410561
1,0.003922,0.604723,0.537194,0.531159,0.526411,1.000000,0.894118,0.937255,0.921569,0.143875,0.091520,0.005778,0.000571,0.003003,0.008353,0.084866,0.521678
2,0.003922,0.735281,0.643212,0.655901,0.623955,1.000000,0.968627,0.945098,0.972549,0.145442,0.061851,0.029713,0.001155,0.002995,0.008177,0.084868,0.418590
3,0.003922,0.669609,0.507003,0.574396,0.513377,1.000000,0.988235,1.000000,0.976471,1.000000,1.000000,0.579690,1.000000,1.000000,1.000000,0.247239,0.266844
4,0.003922,0.515397,0.408786,0.431364,0.420381,0.972549,0.898039,0.858824,0.878431,0.018490,0.000680,0.010861,0.001035,0.003000,0.008482,0.084865,0.304206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,0.003922,0.372645,0.201201,0.260235,0.241906,0.823529,0.701961,0.713725,0.721569,0.222161,0.137293,0.029260,0.008679,0.002973,0.007456,0.085224,0.413292
132,0.003922,0.358751,0.313476,0.324176,0.295171,0.968627,0.984314,1.000000,0.968627,0.291595,0.107894,0.182064,0.377274,0.132540,0.128659,0.104459,0.224758
133,0.003922,0.314633,0.284041,0.212586,0.272055,0.760784,0.780392,0.764706,0.772549,0.206068,0.134252,0.003679,0.000439,0.003003,0.008316,0.084869,0.533983
134,0.003922,0.619880,0.439085,0.467016,0.469161,1.000000,0.937255,0.913725,0.945098,0.150516,0.098095,0.002106,0.000762,0.003005,0.008626,0.084870,0.452198


In [5]:
print(imagenesProcesadas)

['monilia' 'monilia' 'monilia' 'monilia' 'monilia' 'monilia' 'monilia'
 'monilia' 'monilia' 'monilia' 'monilia' 'monilia' 'monilia' 'monilia'
 'monilia' 'monilia' 'monilia' 'monilia' 'monilia' 'monilia' 'monilia'
 'monilia' 'monilia' 'monilia' 'monilia' 'monilia' 'monilia' 'monilia'
 'monilia' 'monilia' 'monilia' 'monilia' 'monilia' 'monilia' 'monilia'
 'monilia' 'monilia' 'monilia' 'monilia' 'monilia' 'monilia' 'monilia'
 'monilia' 'monilia' 'monilia' 'monilia' 'monilia' 'monilia' 'monilia'
 'monilia' 'monilia' 'monilia' 'monilia' 'monilia' 'fito' 'fito' 'fito'
 'fito' 'fito' 'fito' 'fito' 'fito' 'fito' 'fito' 'fito' 'fito' 'fito'
 'fito' 'fito' 'fito' 'fito' 'fito' 'fito' 'fito' 'fito' 'fito' 'fito'
 'fito' 'fito' 'fito' 'fito' 'fito' 'fito' 'fito' 'fito' 'fito' 'fito'
 'fito' 'fito' 'fito' 'fito' 'fito' 'fito' 'fito' 'fito' 'fito' 'fito'
 'fito' 'fito' 'saludable' 'saludable' 'saludable' 'saludable' 'saludable'
 'saludable' 'saludable' 'saludable' 'saludable' 'saludable' 'saludable'

In [8]:
#Generación de tabla con estado del fruto de la imagen y valores de clase para cada imagen del dataset
datasetclase=[]
for i in imagenesProcesadas:
    if i=='fito':
        datasetclase.append(2)

    elif i=='monilia':
        datasetclase.append(1)

    else :
        datasetclase.append(0)

#    datasetclase =[1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,0,0,0,0,0,0,0,0,0,0] #Arreglo de pertenencia a clases del dataset 0=sana 1=moniia 2=fito
#print(datasetclase)
datasetclaseframe = pd.DataFrame({'enfermedad':imagenesProcesadas,'clase':datasetclase})
datasetclaseframe

,enfermedad,clase
0,monilia,1
1,monilia,1
2,monilia,1
3,monilia,1
4,monilia,1
...,...,...
131,saludable,0
132,saludable,0
133,saludable,0
134,saludable,0


En esta ocasión se va a clasificar si el cacao está o no enfermo. 0=sana 1=moniia 2=fito

In [9]:
X = datasetinput.iloc[:,:].values
y = datasetclaseframe.iloc[:,1].values
#X
#y

Ahora separamos los datos en con sub conjuntos, entrenamiento y prueba, dejando el 25% de los registros para prueba y el 75% para entrenamiento, posteriormente ajustamos las escalas.

In [10]:
#Segmentación de datos para entremaiento y validación 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
#Dan 22 de prueba y 8 de validación

# Ajuste de escalas
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
#Conjunto de entrenamiento y conjunto de prueba con ajuste de escalas
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
#X_train
y_train



array([1, 2, 0, 0, 1, 2, 0, 1, 0, 0, 2, 2, 1, 2, 1, 0, 2, 1, 0, 2, 1, 2,
       2, 0, 2, 2, 0, 0, 0, 2, 0, 0, 1, 0, 2, 1, 2, 1, 2, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 2, 2, 0, 2, 2, 2, 1, 0, 2, 2, 1, 1, 2, 0, 1, 0, 1, 0,
       2, 2, 1, 0, 2, 2, 2, 1, 1, 1, 0, 2, 0, 1])

Ahora entrenamos el modelo y predecimos el conjunto X_test. El modelo para el algoritmo KNN lo obtenemos de la clase KNeighborsClassifier de la librería sklearn

Finalmente obtenemos los datos de clasificación a través de la matriz de confusión, en donde los valores verdaderos positivos van por la diagonal y los otros elementos son falsos negativos 

<br>
<img src="../figs/confusionmetrix2.png", width=70%>


In [11]:
# Entrenamiento del modelo KNN
# n_neighbors = numero de vecinos cercanos a tener en cuenta en el algoritmo
# mectrica minkowski = distancia euclidiana

from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)

# Predicción del conjunto de prueba intentando clasificar x_test
y_pred = classifier.predict(X_test)

# Matriz de confusion
# recordar que y_test son los datos ciertos de salida para x_test mientrass qeu y_pred son los datos predichos por modelo para x_test
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm



array([[2, 1, 3],
       [7, 5, 5],
       [4, 3, 4]])

In [102]:
# EFECTIVIDAD DEL 50%
# Visualizacion de los datos de pruebas
## REFERENCIAS
#https://www.youtube.com/watch?v=RBw3hRJmX7I
#https://www.jacobsoft.com.mx/es_mx/k-nearest-neighbors/